# 0) Setup

In [ ]:
import util
import math
import xarray as xr
import numpy as np
import pandas as pd

from blurs import *
from scipy.optimize import minimize

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

# 1) Load the original image

In [ ]:
image = util.load_img('../samples/pumpkins.tif')

fig = px.imshow(image, color_continuous_scale='gray', title='Original Image', width=512, height=512)
fig.update_layout(coloraxis_showscale=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

# 2) Create the point spread function

In [ ]:
psf_params = [3, math.pi/2]
psf_fn = linear
psf = psf_fn(image.shape, *psf_params)

from scipy.fft import ifft2, fftshift
psf_img = np.real(fftshift(ifft2(psf)))

psf_fig = px.imshow(
    psf_img,
    color_continuous_scale='gray', 
    title='Point Spread Function',
    width=512, 
    height=512)
psf_fig.show()

# 3) Create the blurred image

In [ ]:
image_blurred = blur(image, psf)

# optional: add noise
noise_factor = 0.01
image_blurred = noise(image_blurred, noise_factor)

fig = px.imshow(image_blurred, color_continuous_scale='gray', title='Blurred Image', width=512, height=512)
fig.update_layout(coloraxis_showscale=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

# 4) Deblur the image
This assumes that the point source function that was used to generate the blurred image is known. If noise is present when constructing the blurred image, regularization is necessary to compute an approximation of the true image that is not dominated by noise.

This code uses a blurred image and point source function to deblur. The blurred image should be defined in real space, while the point source function in Fourier space.

## Tikhonov Regularization
This deblurring approach filters out values in the point source function according to a regularization parameter $\phi$, where $S$ is the point source function in Fourier space and $\alpha$ is a constant value:

$$\phi = \frac{|S|^2}{|S|^2 + \alpha^2}$$

In the code implementation, a few additional steps are taken to prevent issues with the deblurring attempt. Zero values may exist in $S$ when computing $\phi$, resulting in a divide-by-zero error. This can be remediated by slightly adjusting the zero values to a non-zero number; in this case, the smallest possible number that can be represented in a computer was chosen. The adjustment takes place before $\phi$ is computed, and is only applied to $S$ in the denominator of the equation.

In [ ]:
def deblur_tik(alpha, S, blurred_img):
    # S is fft2(circshift(psf))

    # phi = abs(S).^2 ./ (abs(s).^2 + alpha^2)
    # perform adjustment to values which could result in division by zero
    denominator = np.absolute(np.square(S)) + np.square(alpha)
    denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)
    phi = np.divide(np.absolute(np.square(S)), denominator)

    # filter out S values according to phi 
    S_filt = np.zeros(phi.shape)
    idx = S != 0
    S_filt[idx] = np.divide(phi[idx], S[idx])

    # deblur using filtered S
    X_filt = ifft2(np.multiply(fft2(blurred_img), S_filt)).real

    return X_filt

def deblur_tik_min(S):
    # error function; norm of deblurred image with blurred image
    error_function = lambda input, S, blurred_img: np.linalg.norm(deblur_tik(input[0], S, blurred_img) - image, ord='fro')

    initial_alpha = 0.1
    # since minimizer takes in a vector, need to get first element (which will store alpha)
    result = minimize(error_function, [initial_alpha], args=(S, image_blurred), bounds=[(0, 32)])

    # returns min alpha
    return result

def deblur_psf(psf_params, deblurred_img):
    S_i = linear(image.shape, *psf_params)
    return blur(deblurred_img, S_i)

def error_function(psf_params, deblurred_img, blurred_img):
    result = np.linalg.norm(deblur_psf(psf_params, deblurred_img) - blurred_img, ord='fro')
    return result

def deblur_psf_min(deblurred_img, initial_length):
    # min_{length, angle} = ||A(length, angle) x - b||
    # error_function = lambda psf_params, deblurred_img, blurred_img: np.linalg.norm(deblur_psf(psf_params, deblurred_img) - blurred_img, ord='fro')

    # specifting the solver as 'Powell' was important; otherwise it was stuck at wrong values
    result = minimize(error_function, [initial_length], args=(deblurred_img, image_blurred), method='Powell', bounds=[(0, 16)])

    return result

In [ ]:
# deblur image using various alpha regularization params
# also normalize image values to range [0, 1]
alphas = np.linspace(0, 1, 11)
image_sequence = [util.normalize(deblur_tik(alpha, psf, image_blurred)) for alpha in alphas]

# plot
fig = px.imshow(np.array(image_sequence), animation_frame=0, color_continuous_scale='gray', title='Deblurred Image', width=512, height=512,
                labels=dict(animation_frame="alpha step"))
fig.update_layout(coloraxis_showscale=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

# 5) Deblur using Alternating Minimization Technique

In [ ]:
import time

prev_li = math.nan
li      = 10
prev_ai = math.nan
ai      = 0

images = []
errors = []
psf_lis = []
times   = []

step = 1
# while not math.isclose(prev_li, li) and not math.isclose(prev_ai, ai):
while not math.isclose(prev_li, li):
    start_time = time.time()
    # fix length and angle for guess
    psfi = linear(image.shape, li, ai)
    min_alpha = deblur_tik_min(psfi).x[0]
    deblurred_img = deblur_tik(min_alpha, psfi, image_blurred)

    # fix image for guess
    next_A = deblur_psf_min(deblurred_img, li)
    psf_lis.append(li)
    prev_li = li
    prev_ai = ai
    li = next_A.x[0]

    end_time = time.time()

    images.append(np.real(deblurred_img))

    error = np.linalg.norm(deblurred_img - image, ord='fro') / np.linalg.norm(image, ord='fro')
    errors.append(error)
    times.append(end_time - start_time)
    
    step += 1

fig = px.imshow(np.array(images), animation_frame=0, color_continuous_scale='gray', title='Deblurred Image', width=512, height=512,
                labels=dict(animation_frame="iteration"))
fig.update_layout(coloraxis_showscale=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)

fig.show()

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=3)
i = 1
while i < 7:
    fig.add_trace(px.imshow(util.normalize(images[i-1]), origin='lower').data[0], row=(1 if i <= 3 else 2), col=(((i - 1) % 3) + 1))
    i += 1

# add true image
fig.add_trace(px.imshow(util.normalize(image), origin='lower').data[0], row=2, col=3)

layout = px.imshow(images[0], aspect='equal', color_continuous_scale='gray').layout
fig.layout.coloraxis = layout.coloraxis
fig.update_layout(width=1000, height=600, margin=dict(l=10, r=10, b=10, t=10))
fig.update_yaxes(autorange='reversed')

i = 1
for row in range(2):
    for col in range(3):
        fig.update_xaxes(title_text=f"iteration {i}", row=row+1, col=col+1)
        i += 1
fig.update_xaxes(title_text=f"original image", row=2, col=3)
fig.show()

In [ ]:
# plot of errors

fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(1, 6)], y=errors[0:6],
                    line_shape='spline',
                    line=dict(width=12, color="#444444")))

# fig = px.line()
fig.update_traces(marker_size=32)
fig.update_layout(width=1200, height=600, margin=dict(l=10, r=10, b=10, t=10),
    plot_bgcolor='rgba(0,0,0,0)', font=dict(
        size=24,
    ))
fig.update_xaxes(title_text="Iteration Number", linecolor='black', gridcolor='lightgray', nticks=5)
fig.update_yaxes(title_text="Relative Error between<br>Deblurred and Original Image", linecolor='black', gridcolor='lightgray')
fig.show()

In [ ]:
# plot of times

fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(1, 6)], y=times,
                    line_shape='spline',
                    line=dict(width=12, color="#0000ff")))

print(sum(times))

# fig = px.line()
fig.update_traces(marker_size=32)
fig.update_layout(width=1600, height=1000, margin=dict(l=10, r=10, b=10, t=10),
    plot_bgcolor='rgba(0,0,0,0)', font=dict(
        size=24,
    ))
fig.update_xaxes(title_text="Iteration Number", linecolor='black', gridcolor='lightgray', nticks=5)
fig.update_yaxes(title_text="Iteration Time (s)", linecolor='black', gridcolor='lightgray')
fig.show()

# Plot of Convergence Curve for PSF Error

In [ ]:
psf_matrices = [np.real(fftshift(ifft2(linear(image.shape, l, ai)))) for l in psf_lis]
psf_errors   = [np.linalg.norm(m - psf_img, ord='fro') / np.linalg.norm(psf_img, ord='fro') for m in psf_matrices]

# fig = px.imshow(np.array(psf_matrices), animation_frame=0, color_continuous_scale='gray', title='Deblurred Image', width=512, height=512,
#                 labels=dict(animation_frame="iteration"))
# fig.update_layout(coloraxis_showscale=False)
# fig.update_xaxes(showticklabels=False)
# fig.update_yaxes(showticklabels=False)

# fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(1, 6)], y=psf_errors,
                    line_shape='spline',
                    line=dict(width=12, color="#0000ff")))

print(sum(times))

# fig = px.line()
fig.update_traces(marker_size=32)
fig.update_layout(width=1600, height=1000, margin=dict(l=10, r=10, b=10, t=10),
    plot_bgcolor='rgba(0,0,0,0)', font=dict(
        size=24,
    ))
fig.update_xaxes(title_text="Iteration Number", linecolor='black', gridcolor='lightgray', nticks=5)
fig.update_yaxes(title_text="Iteration Time (s)", linecolor='black', gridcolor='lightgray')
fig.show()

# Plot of Relative Errors when Deblurring

In [ ]:
# # number of samples to take
# num_samples = 20

# lengths = np.linspace(1, 20, num_samples)
# angles  = np.linspace(0, 2*np.pi, num_samples)
# errors  = []

# # error_fn = lambda psf_params: np.linalg.norm(deblur_psf(psf_params, deblurred_img) - image_blurred, ord='fro')

# # iterate over lengths
# for l in lengths:
#     row = []
#     for a in angles:
#         psfi = linear(image.shape, l, a)
#         min_alpha = deblur_tik_min(psfi).x[0]
#         deblurred_img = deblur_tik(min_alpha, psfi, image_blurred)

#         error = np.linalg.norm(blur(images[0], psfi) - image_blurred)
#         row.append(error)
#     errors.append(row)

fig = go.Figure(data=[go.Surface(z=errors, y=lengths, x=angles)])
fig.update_layout(width=1000, height=800,
                  margin=dict(l=0, r=0, b=0, t=0),
                  scene = dict(
                    xaxis_title='Angle (rad)',
                    yaxis_title='Length (pixels)',
                    zaxis_title='Relative Error'))
fig.show()